In [1]:

# Step 1: Load Data from Google Colab
from google.colab import drive
import pandas as pd

In [2]:

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Step 1: Import necessary libraries
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import zipfile
import os
import json
from google.colab import files  # For downloading files


In [4]:

# Step 1: Import necessary libraries
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import zipfile
import os
import json
from google.colab import files  # For downloading files


In [5]:
 #Step 2: Initialize a more accurate translation model
model_name ="shhossain/opus-mt-en-to-bn"  # More accurate model for translation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/969k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/303M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

In [6]:
# Define the translation pipeline
translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang="en_XX", tgt_lang="bn_IN")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
# Step 3: Load your JSON dataset
file_path = '/content/drive/MyDrive/Bangla Image dataset with caption/mscoco_images 10k/mscoco_captions_filtered 10 k.json'  # Replace with the actual path
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Step 4: Define a function to translate captions to Bangla
def translate_to_bangla(text):
    try:
        translated = translator(text, max_length=400)[0]['translation_text']
        print(f"Translating: {text} -> {translated}")  # Debugging print
        return translated
    except Exception as e:
        print(f"Error translating {text}: {e}")
        return text  # Return original text in case of error

In [8]:
# Step 5: Translate all captions in the dataset
for key, value in dataset.items():
    original_caption = value['caption']
    translated_caption = translate_to_bangla(original_caption)
    dataset[key]['caption_bangla'] = translated_caption

# Step 6: Ensure the output directory exists
output_dir = '/mnt/data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 7: Save the translated dataset to a new JSON file
output_json_file = os.path.join(output_dir, 'translated_coco_dataset.json')
with open(output_json_file, 'w', encoding='utf-8') as json_file:
    json.dump(dataset, json_file, ensure_ascii=False, indent=4)

# Step 8: Zip the JSON file
zip_file_path = os.path.join(output_dir, 'translated_coco_dataset.zip')
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(output_json_file, os.path.basename(output_json_file))

print(f"Translation completed. Zip file created at: {zip_file_path}")

# Step 9: Download the zip file to your local machine
files.download(zip_file_path)

Streaming output truncated to the last 5000 lines.
Translating: A bare kitchen in the midst of a paint job. -> রঙের চাকরির মাঝে একটি নিষ্কাশন রান্নাঘর।
Translating: Two giraffes on the grass in a fenced area. -> দু'জন জিরাফ একটি সুরক্ষিত এলাকায় ঘাসে।
Translating: A room with wooden slatted walls adorns two shower knobs, a trash can, a soap dish with soap and a sink. -> কাঠের পাতলা দেওয়ালযুক্ত একটি ঘরে দু'টি গোসল কাঁঠাল, আবর্জনার কাঁচা, সাবানের খাবার, সাবান এবং ডুবে যাওয়াকে সজ্জিত করে।
Translating: A white urinal mounted to a bathroom wall. -> একটি সাদা মূত্র বাথরুম প্রাচীরের দিকে এগিয়ে যায়।
Translating: some people riding some bikes right by some boats  -> কিছু লোক কিছু নৌকা দ্বারা কিছু বাইক চালায়
Translating: A couple of toilets sitting in a  bathroom. -> কয়েকটা টয়লেট বাথরুমে বসে আছে।
Translating: A bathroom sink sitting under a bathroom mirror. -> একটা বাথরুম একটি বাথরুমের আয়নায় বসে ডুবে যায়।
Translating: A man sitting on top of a toilet with a toy on his lap. -> একটা টয়ল

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>